- C'e un problema nel filtering: probabilmente quando la shell esterna si divide in due viene filtrata prima che si possa propagare la label. Questo per colpa di remove long agglomerates. Una soluzione potrebbe essere effettuare quel filtering dopo la label propagation.
- C'è da sistemare la funzione remove long agglomerates perché alcuni agglomerati rimangono per oltre 100 time steps e non vanno rimossi. Devo trovare un modo intelligente per rimuovere gli artefatti di ricostruzione e la parte interna della batteria.

In [1]:
import myfunctions as mf
from skimage.io import imshow
import time as clock
import numpy as np
import napari 
from tqdm import tqdm
from numpy.lib.format import open_memmap
from skimage.measure import label, regionprops
import os

In [2]:
def segment4D(exp, end_time=220, skip180=True, smallest_3Dvolume=20, filtering3D=True, OS='Windows'):
    print(f'\nExp {exp} segmentation started\n')
    # defining the time steps for the current experiment
    start_time = mf.exp_start_time()[mf.exp_list().index(exp)]
    time_steps = range(start_time, end_time+1, 2) if skip180 else range(start_time, end_time+1)
    time_index = range(len(time_steps))
    shape = (len(time_steps), 270, 500, 500)
    hypervolume = open_memmap(os.path.join(mf.OS_path(exp, OS), 'hypervolume.npy'), mode='r')
    hypervolume_mask = open_memmap(os.path.join(mf.OS_path(exp, OS), 'hypervolume_mask.npy'), dtype=np.ushort, mode='w+', shape=shape)

    # dealing with the first volume
    previous_volume = hypervolume[0]
    # evaluating the threshold on the first volume
    threshold = mf.find_threshold(previous_volume)  
    # segmenting the first volume
    print('Segmenting first volume...')
    previous_mask = mf.segment3D(previous_volume, threshold, smallest_volume=smallest_3Dvolume, filtering=filtering3D)
    print('First volume segmentation completed\n')
    # reassigning the labels after the filtering
    labels = [rp.label for rp in regionprops(previous_mask)]
    for new_label, old_label in mf.iterator(enumerate(labels), verbose=True, desc='Renaming labels', leave=False, custom_length=len(labels)):
        previous_mask[previous_mask == old_label] = new_label + 1
    # writing the first mask in the hypervolume_mask memmap
    hypervolume_mask[0] = previous_mask

    # segmenting the remaining volumes and propagating the labels from previous volumes
    for time in tqdm(time_index[1:], desc='Volume segmentation and forward propagation'):
        current_volume = hypervolume[time]
        current_mask = mf.segment3D(current_volume, threshold, smallest_volume=smallest_3Dvolume, filtering=filtering3D)
        current_mask = mf.propagate_labels(previous_mask, current_mask, forward=True, verbose=True)
        # current_mask = mf.remove_long_agglomerates(current_mask)
        hypervolume_mask[time] = current_mask
        previous_mask = current_mask
            
    print(f'\nExp {exp} segmentation completed!\n')
    return hypervolume_mask

In [3]:
exp = mf.exp_list()[0]
OS = 'MacOS'

hypervolume_mask = segment4D(exp=exp, OS=OS)
mf.filtering4D(hypervolume_mask)

# df = mf.motion_df(hypervolume_mask, exp=exp)
# print('Saving properties...')
# df.to_csv(os.path.join(mf.OS_path(exp, OS), 'motion_properties.csv'), index=False)
# print('Done!')


Exp P28A_FT_H_Exp1 segmentation started


Finding threshold...
Threshold=1.81 found in 3.26 s

Segmenting first volume...


First volume segmentation completed



Volume segmentation and forward propagation: 100%|██████████| 54/54 [38:35<00:00, 42.89s/it]



Exp P28A_FT_H_Exp1 segmentation completed!


4D filtering started



Renaming labels: 100%|██████████| 55/55 [10:21<00:00, 11.30s/it]


4D filtering completed!



In [4]:
hypervolume = open_memmap(os.path.join(mf.OS_path(exp, OS), 'hypervolume.npy'), mode='r')
# hypervolume_mask = open_memmap(os.path.join(mf.OS_path(exp, OS), 'hypervolume_mask.npy'), mode='r')

In [5]:
print(hypervolume_mask.shape)
print(hypervolume.shape)

(55, 270, 500, 500)
(55, 270, 500, 500)


In [6]:
viewer = napari.Viewer()

images = [viewer.add_image(hypervolume, name='Volume', opacity=0.4)]
labels = [viewer.add_labels(hypervolume_mask, name='Labels', blending='additive', opacity=0.8)]

settings = napari.settings.get_settings()
settings.application.playback_fps = 5
viewer.dims.current_step = (0, 0)

In [50]:
tic = clock.perf_counter()
unique_labels = np.unique(hypervolume_mask[0])
print(f'np.unique: {clock.perf_counter()-tic:.6f}s')
tic = clock.perf_counter()
labels = [rp.label for rp in regionprops(hypervolume_mask[0])]
print(f'regionprops: {clock.perf_counter()-tic:.6f}s')

np.unique: 0.577486s
regionprops: 0.149048s


In [52]:
sequence_mask = hypervolume_mask[0]
slices = range(sequence_mask.shape[0])
tic = clock.perf_counter()
labels = [np.unique(sequence_mask[z]) for z in slices]
print(f'np.unique: {clock.perf_counter()-tic:.6f}s')
tic = clock.perf_counter()
labels = [[rp.label for rp in regionprops(sequence_mask[z])] for z in slices]
print(f'regionprops: {clock.perf_counter()-tic:.6f}s')

np.unique: 0.378467s
regionprops: 0.230809s
